In [ ]:
!pip install datasets
!pip install imblearn

In [ ]:
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

dataset_name = "OsBaran/tryTr5"  
dataset = load_dataset(dataset_name)

df = pd.DataFrame(dataset['train'])
# None veya NaN değerler
print("Eksik veri sayısı:\n", df.isnull().sum())

df = df.dropna(subset=['text', 'label'])

print("Temizleme sonrası boyut:", df.shape)

print(df['label'].value_counts())

balanced_train_data = Dataset.from_pandas(df)
dataset_name = "OsBaran/tryTr4"  

dataset = load_dataset(dataset_name)

df = pd.DataFrame(dataset['train'])
print("Eksik veri sayısı:\n", df.isnull().sum())

df = df.dropna(subset=['text', 'label'])

print("Temizleme sonrası boyut:", df.shape)

print(df['label'].value_counts())

test_data = Dataset.from_pandas(df)
# test_data = load_dataset("OsBaran/tryTr5")["train"]
# train_data, test_data = train_test_split(df, test_size=0, stratify=df['label'], random_state=42)

# print("Eğitim Seti Dağılımı:\n", train_data['label'].value_counts(normalize=True))
# print("Test Seti Dağılımı:\n", test_data['label'].value_counts(normalize=True))

# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler(random_state=42)
# balanced_train_data, balanced_train_labels = ros.fit_resample(
#     train_data[['text']], train_data['label']
# )

# balanced_train_data = pd.DataFrame(balanced_train_data, columns=["text"])
# balanced_train_data["label"] = balanced_train_labels

# print("Dengeli Eğitim Seti Dağılımı:\n", balanced_train_data['label'].value_counts())
test_data

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(balanced_train_data["text"])

X_train_seq = tokenizer.texts_to_sequences(balanced_train_data["text"])
X_test_seq = tokenizer.texts_to_sequences(test_data["text"])

max_length = 2000  # Maksimum sekans uzunluğu
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding="post")
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding="post")
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(balanced_train_data["label"])
y_test = label_encoder.transform(test_data["label"])

model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length), 
    LSTM(128, return_sequences=False),  # LSTM katmanı
    Dropout(0.3),  
    Dense(64, activation="relu"),  # Ara katman
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # Çıkış katman
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# eğitme
model.fit(X_train, np.array(y_train), epochs=5, batch_size=32, validation_split=0.2)

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")


print("Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred))

# Karışıklık matrisi
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Olumsuz", "Olumlu"], yticklabels=["Olumsuz", "Olumlu"])
plt.xlabel("Tahmin Edilen")
plt.ylabel("Gerçek")
plt.title("Karışıklık Matrisi")
plt.show()
